# SOIL Tutorial

This notebook contains a tutorial to learn how to use the SOcial network sImuLator (SOIL) written in Python. 

## Introduction

SOIL is based in 3 main files:
* __soil.py__: It's the main file of SOIL. The network creation, simulation and visualization are done in this file.
- __models.py__: All the spread models already implemented are stored in this file.
+ __settings.py__: This file contains every variable needed in the simulation in order to be modified easily.

## Soil.py

### Imports and data initialization

First of all, you need to make all the imports. This simulator is based on [nxsim](https://pypi.python.org/pypi/nxsim), using [networkx](https://networkx.github.io/) for network management. We will also include the models and settings files where the spread models and initialization variables are stored.

In [25]:
import matplotlib.pyplot as plt
from nxsim import NetworkSimulation
import numpy
import networkx as nx
import settings
import models
from models import *
import math
import json

settings.init() # Loads all the data from settings
models.init() # Loads the models and network variables

ImportError: No module named 'matplotlib'

### Network creation

Using a parameter provided in the settings file, we can choose what type of network we want to create, as well as the number of nodes and some other parameters. More types of networks can be implemented using [networkx](https://networkx.github.io/).

In [ ]:
if settings.network_type == 0:
    G = nx.complete_graph(settings.number_of_nodes)
if settings.network_type == 1:
    G = nx.barabasi_albert_graph(settings.number_of_nodes,10)
if settings.network_type == 2:
    G = nx.margulis_gabber_galil_graph(settings.number_of_nodes, None)
# More types of networks can be added here

### Simulation

The simulation starts with the following code. The user can provide the network topology, the maximum time of simulation, the spread model to be used as well as other parameters.

In [ ]:
sim = NetworkSimulation(topology=G, states=init_states, agent_type=ControlModelM2,
                        max_time=settings.max_time, num_trials=settings.num_trials, logging_interval=1.0)


sim.run_simulation()

### Visualization

In order to analyse the results of the simulation. We include them in the topology and a .gexf file is generated. This allows the user to picture the network in [Gephi](https://gephi.org/). A JSON file is also generated to permit further analysis.

This is done with the following code:

In [ ]:
for x in range(0, settings.number_of_nodes):
    for empresa in models.networkStatus["agente_%s"%x]:
        emotionStatusAux=[]
        for tiempo in models.networkStatus["agente_%s"%x][empresa]:
            prec = 2
            output = math.floor(models.networkStatus["agente_%s"%x][empresa][tiempo] * (10 ** prec)) / (10 ** prec) #Para tener 2 decimales solo
            emotionStatusAux.append((output,tiempo,None))
        attributes = {}
        attributes[empresa] = emotionStatusAux
        G.add_node(x, attributes)


print("Done!")

with open('data.txt', 'w') as outfile:
    json.dump(models.networkStatus, outfile, sort_keys=True, indent=4, separators=(',', ': '))

nx.write_gexf(G,"test.gexf", version="1.2draft")

That's only the basic visualization. Everything you need can be implemented as well. For example:

In [ ]:
x_values = []
infected_values = []
neutral_values = []
cured_values = []
vaccinated_values = []

attribute_plot = 'status'
for time in range(0, settings.max_time):
    value_infectados = 0
    value_neutral = 0
    value_cured = 0
    value_vaccinated = 0
    real_time = time * settings.timeout
    activity= False
    for x in range(0, settings.number_of_nodes):
        if attribute_plot in models.networkStatus["agente_%s" % x]:
            if real_time in models.networkStatus["agente_%s" % x][attribute_plot]:
                if models.networkStatus["agente_%s" % x][attribute_plot][real_time] == 1: ##Representar infectados
                    value_infectados += 1
                    activity = True
                if models.networkStatus["agente_%s" % x][attribute_plot][real_time] == 0:  ##Representar neutrales
                    value_neutral += 1
                    activity = True
                if models.networkStatus["agente_%s" % x][attribute_plot][real_time] == 2:  ##Representar cured
                    value_cured += 1
                    activity = True
                if models.networkStatus["agente_%s" % x][attribute_plot][real_time] == 3:  ##Representar vaccinated
                    value_vaccinated += 1
                    activity = True

    if activity:
        x_values.append(real_time)
        infected_values.append(value_infectados)
        neutral_values.append(value_neutral)
        cured_values.append(value_cured)
        vaccinated_values.append(value_vaccinated)
        activity=False

infected_line = plt.plot(x_values,infected_values,label='Infected')
neutral_line = plt.plot(x_values,neutral_values, label='Neutral')
cured_line = plt.plot(x_values,cured_values, label='Cured')
vaccinated_line = plt.plot(x_values,vaccinated_values, label='Vaccinated')
plt.legend()
plt.savefig('control_model.png')

## Models.py

### Imports and initialization

In [ ]:
from nxsim import BaseNetworkAgent
import numpy as np
import random
import settings

settings.init()

##############################
# Variables initializitation #
##############################
def init():
    global networkStatus
    networkStatus = {}  # Dict that will contain the status of every agent in the network

sentimentCorrelationNodeArray=[]
for x in range(0, settings.number_of_nodes):
    sentimentCorrelationNodeArray.append({'id':x})
# Initialize agent states. Let's assume everyone is normal.
init_states = [{'id': 0, } for _ in range(settings.number_of_nodes)]  # add keys as as necessary, but "id" must always refer to that state category

### Base behaviour

Every spread model used in SOIL should extend the base behaviour class.  By doing this the exportation of the attributes values will be automatic. This feature will be explained in the Spread Models section. The class looks like this:

In [ ]:
class ComportamientoBase(BaseNetworkAgent):
    def __init__(self, environment=None, agent_id=0, state=()):
        super().__init__(environment=environment, agent_id=agent_id, state=state)
        self._attrs = {}

    @property
    def attrs(self):
        now = self.env.now
        if now not in self._attrs:
            self._attrs[now] = {}
        return self._attrs[now]

    @attrs.setter
    def attrs(self, value):
        self._attrs[self.env.now] = value

    def run(self):
        while True:
            self.step(self.env.now)
            yield self.env.timeout(settings.timeout)

    def step(self, now):
        networkStatus['agente_%s'% self.id] = self.a_json()

    def a_json(self):
        final = {}
        for stamp, attrs in self._attrs.items():
            for a in attrs:
                if a not in final:
                   final[a] = {}
                final[a][stamp] = attrs[a]
        return final


### Spread models

Every model to be implemented must include an init and a step function.  Depending on your model, you would need different attributes. If you want them to be automatic exported for a further analysis, you must name them like this *self.attrs['name_of_attribute']*. Moreover, the last thing you should do inside the step function is call the following method *super().step(now)*. This call will store the values.

Some other tips:
* __self.state['id']__: To check the id of the current agent/node.
* __self.get_neighboring_agents(state_id=x)__: Returns the neighbours agents/nodes with the id provided

An example of a spread model already implemented and working:



In [ ]:
class ControlModelM2(ComportamientoBase):
    #Init infected
    init_states[random.randint(0,settings.number_of_nodes-1)] = {'id':1}
    init_states[random.randint(0,settings.number_of_nodes-1)] = {'id':1}

    # Init beacons
    init_states[random.randint(0, settings.number_of_nodes-1)] = {'id': 4}
    init_states[random.randint(0, settings.number_of_nodes-1)] = {'id': 4}
    def __init__(self, environment=None, agent_id=0, state=()):
        super().__init__(environment=environment, agent_id=agent_id, state=state)

        self.prob_neutral_making_denier = np.random.normal(settings.prob_neutral_making_denier, settings.standard_variance)

        self.prob_infect = np.random.normal(settings.prob_infect, settings.standard_variance)

        self.prob_cured_healing_infected = np.random.normal(settings.prob_cured_healing_infected, settings.standard_variance)
        self.prob_cured_vaccinate_neutral = np.random.normal(settings.prob_cured_vaccinate_neutral, settings.standard_variance)

        self.prob_vaccinated_healing_infected = np.random.normal(settings.prob_vaccinated_healing_infected, settings.standard_variance)
        self.prob_vaccinated_vaccinate_neutral = np.random.normal(settings.prob_vaccinated_vaccinate_neutral, settings.standard_variance)
        self.prob_generate_anti_rumor = np.random.normal(settings.prob_generate_anti_rumor, settings.standard_variance)

    def step(self, now):

        if self.state['id'] == 0:  #Neutral
            self.neutral_behaviour()
        elif self.state['id'] == 1:  #Infected
            self.infected_behaviour()
        elif self.state['id'] == 2:  #Cured
            self.cured_behaviour()
        elif self.state['id'] == 3:  #Vaccinated
            self.vaccinated_behaviour()
        elif self.state['id'] == 4:  #Beacon-off
            self.beacon_off_behaviour()
        elif self.state['id'] == 5:  #Beacon-on
            self.beacon_on_behaviour()

        self.attrs['status'] = self.state['id']
        super().step(now)


    def neutral_behaviour(self):

        # Infected
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        if len(infected_neighbors)>0:
            if random.random() < self.prob_neutral_making_denier:
                self.state['id'] = 3   # Vaccinated making denier

    def infected_behaviour(self):

        # Neutral
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_infect:
                neighbor.state['id'] = 1  # Infected

    def cured_behaviour(self):

        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

        # Cure
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_cured_healing_infected:
                neighbor.state['id'] = 2  # Cured


    def vaccinated_behaviour(self):

        # Cure
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_cured_healing_infected:
                neighbor.state['id'] = 2  # Cured


        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

        # Generate anti-rumor
        infected_neighbors_2 = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors_2:
            if random.random() < self.prob_generate_anti_rumor:
                neighbor.state['id'] = 2  # Cured

    def beacon_off_behaviour(self):
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        if len(infected_neighbors) > 0:
            self.state['id'] == 5  #Beacon on

    def beacon_on_behaviour(self):

        # Cure (M2 feature added)
        infected_neighbors = self.get_neighboring_agents(state_id=1)
        for neighbor in infected_neighbors:
            if random.random() < self.prob_generate_anti_rumor:
                neighbor.state['id'] = 2  # Cured
            neutral_neighbors_infected = neighbor.get_neighboring_agents(state_id=0)
            for neighbor in neutral_neighbors_infected:
                if random.random() < self.prob_generate_anti_rumor:
                    neighbor.state['id'] = 3  # Vaccinated
            infected_neighbors_infected = neighbor.get_neighboring_agents(state_id=1)
            for neighbor in infected_neighbors_infected:
                if random.random() < self.prob_generate_anti_rumor:
                    neighbor.state['id'] = 2  # Cured


        # Vaccinate
        neutral_neighbors = self.get_neighboring_agents(state_id=0)
        for neighbor in neutral_neighbors:
            if random.random() < self.prob_cured_vaccinate_neutral:
                neighbor.state['id'] = 3  # Vaccinated

## Settings.py

This file contains all the variables that can be modified from the simulation. In case of implementing a new spread model, the new variables should be also included in this file.

In [ ]:
# settings.py
def init():

    network_type=1
    number_of_nodes=1000
    max_time=50
    num_trials=1
    timeout=2

    #Zombie model
    bite_prob=0.01 # 0-1
    heal_prob=0.01 # 0-1

    #Bass model
    innovation_prob=0.001
    imitation_prob=0.005

    #Sentiment Correlation model
    outside_effects_prob = 0.2
    anger_prob = 0.06
    joy_prob = 0.05
    sadness_prob = 0.02
    disgust_prob = 0.02

    #Big Market model
    ##Names
    enterprises = ["BBVA","Santander", "Bankia"]
    ##Users
    tweet_probability_users = 0.44
    tweet_relevant_probability = 0.25
    tweet_probability_about = [0.15, 0.15, 0.15]
    sentiment_about = [0, 0, 0] #Valores por defecto
    ##Enterprises
    tweet_probability_enterprises = [0.3, 0.3, 0.3]

    #SISa
    neutral_discontent_spon_prob = 0.04
    neutral_discontent_infected_prob = 0.04
    neutral_content_spon_prob = 0.18
    neutral_content_infected_prob = 0.02

    discontent_neutral = 0.13
    discontent_content = 0.07
    variance_d_c = 0.02

    content_discontent = 0.009
    variance_c_d = 0.003
    content_neutral = 0.088

    standard_variance = 0.055

    #Spread Model M2 and Control Model M2
    prob_neutral_making_denier = 0.035

    prob_infect = 0.075

    prob_cured_healing_infected = 0.035
    prob_cured_vaccinate_neutral = 0.035

    prob_vaccinated_healing_infected = 0.035
    prob_vaccinated_vaccinate_neutral = 0.035
    prob_generate_anti_rumor = 0.035
